In [1]:
import pandas as pd
import numpy as np
import util

enrollment_path = './data/tn/enrollment/'

years = [
    2013
    , 2014
    , 2015
    , 2016
    , 2017
    , 2018
    , 2019
]

enrollment_files = [
    'tn_enrollment_2013_cleaned.csv'
    , 'tn_enrollment_2014_cleaned.csv'
    , 'tn_enrollment_2015_cleaned.csv'
    , 'tn_enrollment_2016_cleaned.csv'
    , 'tn_enrollment_2017_cleaned.csv'
    , 'tn_enrollment_2018_cleaned.csv'
    , 'tn_enrollment_2019_cleaned.csv'
]

df = pd.read_csv(enrollment_path + enrollment_files[0])
df['year'] = df.apply(lambda x: years[0], axis=1)

for i in range(1, len(enrollment_files)):
    df2 = pd.read_csv(enrollment_path + enrollment_files[i])
    df2['year'] = df2.apply(lambda x: years[i], axis=1)
    df = df.append(df2, ignore_index = True)

   district_id         district  school_id                       school  \
0           10  Anderson County          2  Anderson County High School   
1           10  Anderson County          2  Anderson County High School   
2           10  Anderson County          2  Anderson County High School   
3           10  Anderson County          2  Anderson County High School   
4           10  Anderson County          2  Anderson County High School   

        grade                group_state gender   num  year  
0  All Grades     All Race/Ethnic Groups  M & F  1030  2013  
1  All Grades                      Asian      F     1  2013  
2  All Grades  Black or African American      M     5  2013  
3  All Grades            Hispanic/Latino      F     1  2013  
4  All Grades            Hispanic/Latino      M     1  2013  


In [5]:
# rename All Race/Ethnic Groups to All Groups
df['temp'] = [x if x != 'All Race/Ethnic Groups' else 'All Groups' for x in df['group_state']]
del df['group_state']
df = df.rename(columns={"temp": "group_state"})

# drop rows with combined genders
df = df[df['gender'] != 'M & F']
df = df[df['gender'] != 'All Genders']

0                            All Race/Ethnic Groups
1                                             Asian
2                         Black or African American
3                                   Hispanic/Latino
5                                 Two or more races
6                                             White
265                American Indian or Alaska Native
804       Native Hawaiian or Other Pacific Islander
432178            American Indian or Alaskan Native
432185                 Hispanic or Latino ethnicity
472305                                  No response
747240                                     Hispanic
747246          Native Hawaiian or Pacific Islander
977538                          Multiple Ethinicity
Name: group_state, dtype: object
0               M & F
1                   F
2                   M
288189    All Genders
Name: gender, dtype: object
1                                             Asian
2                         Black or African American
3                  

In [7]:
df.columns
# roll up values to get rid of gender detail
df = df.groupby(['year', 'district_id', 'district', 'school_id', 'school', 'grade', 'group_state'], as_index=False)['num'].sum()

In [9]:
# import 2012 values (All Grades only)
df_2012 = pd.read_csv(enrollment_path + 'tn_enrollment_2012_nogrades_cleaned.csv')

In [10]:
# transpose 2012 data
df_2012 = pd.melt(df_2012, id_vars=['year', 'district_id', 'district', 'school_id', 'school']
        , value_vars=['All Groups'
            , 'White'
            , 'African American'
            , 'Hispanic'
            , 'Asian'
            , 'Native American'
            , 'Free eligible'
            , 'Reduced eligible'
            , 'Free reduced eligible'
            ],
        var_name='group_state'
        , value_name='num')

# add grade column
df_2012['grade'] = df_2012.apply(lambda x: 'All Grades', axis=1)

In [12]:
 df = df.append(df_2012, ignore_index = True, sort=True)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [14]:
df.to_csv('./data/finalized/tn_enrollment.csv', index=False)